In [35]:
import re
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
from langdetect import detect

In [36]:
jobs = pd.read_csv('processed_data/jobs_data.csv')

In [37]:
def search_fork(text):
    text = text.lower()
    regexs = [r'вилк', r'зп', r'з/п', r'з\п', r'зарплат', r'заработн', r'оплат',
              r'компенсац', r'доход', 'fork:', r'moneys', r'salary', r'moneyparrot:', r'деньг', r'оклад', r'условия:', 
              r'ставка', r'_plug:']
    i = 0
    salary = None
    nums = None
    while i<len(regexs) and salary is None :
        salary, nums = search_keyword(regexs[i], text)
        if nums is None: 
            salary = None
            salary, nums = search_keyword(regexs[i], text, True)
            if nums is None: salary = None
        i += 1
    return salary, nums

In [38]:
def search_keyword(word, text, with_ent=False):
    finder = re.search(word, text)
    salary = None
    nums = None
    if finder:
        start = finder.start()
        salary = ''
        text_s = text[start:]
        for i, ch in enumerate(text_s):
            if ch != '\n' and i != len(text_s)-1:
                salary += ch
            else:
                if with_ent == False:
                    nums = search_numbers_in_salary_text(salary)
                    return salary, nums
                else:
                    salary += ch
                    with_ent = False
    return salary, nums

In [61]:
def search_numbers_in_salary_text(text):
    if text is None: return None
    nums = []
    num = ''
    for ch in text:
        if ch.isdigit() or ch in ('.'):
            if len(num)==0 and ch=='0' and len(nums)>0:
                num = nums.pop()
            num += ch
        else:
            if len(num)>0:
                try: 
                    float(num)
                    nums.append(num)
                except Exception as e:
                    pass
            num = ''
            
    if len(num)>0 and num!='.': nums.append(num)
    return nums if nums else None

In [62]:
def search_for_salary(text):
    regex = r'\d{2,6}[-докКтысрщ.]{1,4}\d{2,6}\s?(?:к|К|тыс|тысяч|т\s?р|т.\s?р.|gross|net)?'
    # Почему-то неправильная регулярка:(
    # regex = r'\d{2,6}[-докКтысрщ.](?:{1,4}\d{2,6}\s?(?:к|К|тыс|тысяч|т\s?р|т.\s?р.|gross|net)?)?'
    result = re.findall(regex, text, flags=re.IGNORECASE)
    return result


def get_salaries(data):
    salary = []
    nums = []
    is_eng = []
    for d in tqdm(data.itertuples()):
        if detect(d.text) != 'ru':
            is_eng.append(1)
        else:
            is_eng.append(0)
        res, num = search_fork(d.text)
        if nums is not None:
            nums.append(num)
            salary.append(res)
#         ids.append(i)
    return salary, nums, is_eng

In [63]:
salary, nums, is_eng = get_salaries(jobs)

<ipython-input-62-1b40958a7211>:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for d in tqdm(data.itertuples()):


In [64]:
jobs['salary'] = salary
jobs['nums'] = nums
jobs['is_eng'] = is_eng

### Проверка парсинга

In [65]:
# процент вакансий с нераспаршенными зп
100*jobs['salary'].isnull().sum()/jobs.shape[0], jobs['salary'].isnull().sum()

(16.42542479546885, 522)

In [66]:
jobs[['salary', 'nums', 'text']].tail(30)#['text'][2]

,salary,nums,text
3148,вилка* - 160-250,"[160, 250]",*вакансия: devops (системный администратор)*\n...
3149,вилка: *150 000 – 250 000* руб/мес,"[150000, 250000]",компания: :sber_new:*сбербанк*\nгород: москва...
3150,вилка: *150 000 – 250 000* руб/мес,"[150000, 250000]",компания: :sber_new:*сбербанк*\nгород: москва ...
3151,None,None,*senior software engineer (python/devops/data ...
3152,вилка: нетто* 60k - 100k rub/month + годовой б...,"[60, 100, 2]",*название вакансии:* project manager команды а...
3153,вилка:* <tel:144 000 -180000|144 000 -180000> ...,"[144000, 180000, 144000, 180000, 2, 4]",всем привет!\n*компания*: сбербанк\n*локация*:...
3154,ставка может обсуждаться индивидуально! от 400...,"[4000, 1, 5]",преподаватель на онлайн курс data engineer для...
3155,вилка: gross*\n• middle: 150 - 200к + годовой ...,"[150, 200, 2, 5]",всем привет!\nкомпания: сбербанк.\n*локация*: ...
3156,вилка 150k-250k ₽ (net),"[150, 250]",:spock-hand:\n*photo lab* - <https://pho.to>...
3157,вилка:* 80 - 120к net,"[80, 120]",*роль:* data engineer\n*компания:* <http://www...


In [68]:
jobs[jobs['is_eng']==1][['salary', 'nums', 'text']].tail(30)

,salary,nums,text
2994,вилка: 2000$ - 3500$*,"[2000, 3500]",*вторая:* :flag-ua:\n\n*компания: parimatch uk...
2999,salary range :* 120-200+ gross (+bonus 20%),"[120, 200, 20]",*job title* : junior+/middle data scientist \n...
3007,salary_:* up to 52k€ + bonuses,[52],"hi everyone,\n\ndiagnoly is looking for a seni..."
3014,None,None,```\nвсем привет!\nк нам в kaspersky lab ищем ...
3027,None,None,scalarr is an international mobile anti-fraud ...
3029,salary: * 140 000 - 200 000 rub/monthly,"[140000, 200000]",hello!\ni'm a recruiter in prophecy labs and w...
3037,None,None,всем привет! в команду развития бизнеса nvidia...
3040,"зп (от 180к до 240к рублей net, с интересным к...","[180, 240]",всем привет!\nищем: data engineer\nпроект: мас...
3049,fork:: 12-16k $ (это моя оценка на основе знан...,"[12, 16, 12]","от себя:\n- контору знаю в которую ищут, но го..."
3050,вилка: 120 - 150 000 рублей gross,"[120, 150000]",вакансии: data analyst\nгород: москва\nкомпани...


In [31]:
jobs[(jobs.nums.isnull())&(jobs.is_eng==0)][['text', 'salary']].tail(20)

,text,salary
2687,и снова привет! ещё две вакансии от *astral ai...,None
2701,"*компания:* лента :lenta:, big data department...",None
2751,подарок на нг)\nвакансия фулстак софтверщика в...,None
2778,всем привет! :wave:\n\n*после фидбека от ods п...,None
2784,всем привет!\nмы в *<https://www.semrush.com/|...,None
2870,"привет ods. меня зовут паша, я научный сотрудн...",None
2878,*вакансия: lead nlp researcher/tech lead в лаб...,None
2880,• опыт работы от 3 лет в сфере обработки аудио...,None
2881,всем привет:wave:\n\nнаша data science-команда...,None
2911,после первых публикаций к нам уже пришли класс...,None


#### Надо придумать регулярку для парсинга вакансий без ключевых слов

In [263]:
jobs['text'][2687]

'и снова привет! ещё две вакансии от *astral ai*, на этот раз в медицинскую команду.\n*компания*: astral ai\n*команда*: <http://celsus.ai|цельс>\n*локация*: петербург, невский проспект\n*занятость*: full-time\n*позиция 1*: специалист в области computer vision (middle/senior), 130-180 middle, 180-300 senior (net)\n*позиция 2*: аналитик по медицинским данным (junior), 60-120 (net)\n\n*мы ищем тех, кто*\n• хочет работать в энергичной, слегка чокнутой команде\n• не боится рисковать и принимать важные решения\n• умеет слушать чужие мнения и не стесняется высказывать своё\n• готов работать в условиях не до конца сформированного, хаотичного рынка медицинских ai-технологий\n• не падает в обморок при виде рентгеновских снимков и готов регулярно впитывать сакральные медицинские знания от врачей\n*наши проекты:*\n• маммография - детектирование злокачественных и доброкачественных образований на снимках молочной железы. мы - первая команда, прошедшая в продуктовый контур эксперимента по использован